# 读取YOLOv3检测结果

In [1]:
import os
import pandas as pd
import numpy as np

from PIL import Image
from vizer.draw import draw_boxes
from numpy import dot
from scipy.linalg import inv, block_diag



rct_file = "./city/test/comp4_det_test_stopline.txt"
img_list = []
cls_score = []
bbox = []
with open(rct_file, 'r') as f: 
    for line in f.readlines():
        img_list.append('./image/'+line.split()[0]+'.bmp')
        cls_score = float(line.split()[1][:5])
        xmin = int(float(line.split()[2]))
        ymin = int(float(line.split()[3]))
        xmax = int(float(line.split()[4]))
        ymax = int(float(line.split()[5]))
        bbox.append([cls_score,xmin,ymin,xmax,ymax])
f.close()


dic = dict(zip(img_list,[[] for _ in range(len(img_list))]))

for i, image_path in enumerate(img_list):
    dic[image_path].append(bbox[i])

det_name = []
for i in range(7050,8072):
    det_name.append('./image/frame_vc1_' + str(i) + '.bmp')



In [2]:
import numpy as np
import cv2 as cv
from numpy import shape
import random
import copy
import matplotlib.pyplot as plt


def box_iou2(a, b):
    '''
    Helper funciton to calculate the ratio between intersection and the union of
    two boxes a and b
    a[0], a[1], a[2], a[3] <-> left, up, right, bottom
    '''
    
    w_intsec = np.maximum (0, (np.minimum(a[2], b[2]) - np.maximum(a[0], b[0])))
    h_intsec = np.maximum (0, (np.minimum(a[3], b[3]) - np.maximum(a[1], b[1])))
    s_intsec = w_intsec * h_intsec
    s_a = (a[2] - a[0])*(a[3] - a[1])
    s_b = (b[2] - b[0])*(b[3] - b[1])
    
    return float(s_intsec)/(s_a + s_b -s_intsec + 1e-16)


dic1 = {}
# dic1 = dict(zip(det_name, [dic[keys] for keys in det_name]))    
dic1 = dic.copy()

for i in det_name:
    if i not in dic.keys():
        dic1[i] = [[0,0,0,0,0]]
        
z_box = []

for i, img in enumerate(det_name):
    boxes = dic1[img]
    score = []
    b_box = []
    boxes = np.array(boxes, dtype=float)
    index = boxes[0:,0].argmax()
    if img == "./image/frame_vc1_7050.bmp":
        b_box.append(boxes[index,1:])
        z_box.append(boxes[index,1:])

    if boxes[index,0] > 0.3:
        b_box.append(boxes[index,1:])
        z_box.append(boxes[index,1:])
    else:
        z_box.append([0, 0, 0, 0])
#     elif boxes[index,0] > 0.3:
#         score.append(boxes[index,0])
#         b_box.append(boxes[index,1:])
#         z_box.append(boxes[index,1:])


#     for i in range(len(boxes)):
#         if boxes[i, 0] > 0.2 and i!=index:
#             print(img,boxes[i,0], boxes[index,0])
#             print(boxes[i,1:], boxes[index,1:])
#             iou_score = box_iou2(boxes[i,1:], boxes[index,1:])
#             print(iou_score)
#             if iou_score < 0.05:
#                 score.append(boxes[i,0])
#                 b_box.append(boxes[i,1:])
#                 z_box.append(boxes[index,1:])

    image_name = os.path.basename(img)
    image = np.array(Image.open(img).convert("RGB"))
    drawn_image = draw_boxes(image, b_box, None, None, None).astype(np.uint8)
    Image.fromarray(drawn_image).save(os.path.join('./test_img', image_name))


# 定义 Kalman Filter 跟踪器

In [3]:
class Tracker(): # class for Kalman Filter-based tracker
    def __init__(self):
        # Initialize parametes for tracker (history)
        self.id = 0  # tracker's id 
        self.box = [] # list to store the coordinates for a bounding box 
        self.hits = 0 # number of detection matches
        self.no_losses = 0 # number of unmatched tracks (track loss)
        # Initialize parameters for Kalman Filtering
        # The state is the (x, y) coordinates of the detection box
        # state: [up, up_dot, left, left_dot, down, down_dot, right, right_dot]
        # or[up, up_dot, left, left_dot, height, height_dot, width, width_dot]
        self.x_state=[] 
        self.dt = 1.   # time interval
        
        # Process matrix, assuming constant velocity model
        self.F = np.array([[1, self.dt, 0,  0,  0,  0,  0, 0],
                           [0, 1,  0,  0,  0,  0,  0, 0],
                           [0, 0,  1,  self.dt, 0,  0,  0, 0],
                           [0, 0,  0,  1,  0,  0,  0, 0],
                           [0, 0,  0,  0,  1,  self.dt, 0, 0],
                           [0, 0,  0,  0,  0,  1,  0, 0],
                           [0, 0,  0,  0,  0,  0,  1, self.dt],
                           [0, 0,  0,  0,  0,  0,  0,  1]])
        
        # Measurement matrix, assuming we can only measure the coordinates
        
        self.H = np.array([[1, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 1, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 1, 0, 0, 0], 
                           [0, 0, 0, 0, 0, 0, 1, 0]])
        
        
        # Initialize the state covariance
        self.L = 10.0
        self.P = np.diag(self.L*np.ones(8))
        
        
        # Initialize the process covariance
        self.Q_comp_mat = np.array([[self.dt**4/4., self.dt**3/2.],
                                    [self.dt**3/2., self.dt**2]])
        self.Q = block_diag(self.Q_comp_mat, self.Q_comp_mat, 
                            self.Q_comp_mat, self.Q_comp_mat)
        
        # Initialize the measurement covariance
        self.R_scaler = 1.0
        self.R_diag_array = self.R_scaler * np.array([self.L, self.L, self.L, self.L])
        self.R = np.diag(self.R_diag_array)
        
        
    def update_R(self):   
        R_diag_array = self.R_scaler * np.array([self.L, self.L, self.L, self.L])
        self.R = np.diag(R_diag_array)
        
    def kalman_filter(self, z): 
        '''
        Implement the Kalman Filter, including the predict and the update stages,
        with the measurement z
        '''
        x = self.x_state
        # Predict
        x = dot(self.F, x)
        self.P = dot(self.F, self.P).dot(self.F.T) + self.Q

        #Update
        S = dot(self.H, self.P).dot(self.H.T) + self.R
        K = dot(self.P, self.H.T).dot(inv(S)) # Kalman gain
        y = z - dot(self.H, x) # residual
        x += dot(K, y)
        self.P = self.P - dot(K, self.H).dot(self.P)
        self.x_state = x.astype(int) # convert to integer coordinates 
                                     #(pixel values)
        
    def predict_only(self):  
        '''
        Implment only the predict stage. This is used for unmatched detections and 
        unmatched tracks
        '''
        x = self.x_state 
        # Predict
        x = dot(self.F, x)
        self.P = dot(self.F, self.P).dot(self.F.T) + self.Q
        self.x_state = x.astype(int)
        
from sklearn.utils.linear_assignment_ import linear_assignment

def Giou_np(bbox_p, bbox_g):
    """
    :param bbox_p: predict of bbox(N,4)(x1,y1,x2,y2)
    :param bbox_g: groundtruth of bbox(N,4)(x1,y1,x2,y2)
    :return:
    """
    
    # for details should go to https://arxiv.org/pdf/1902.09630.pdf
    # ensure predict's bbox form
    x1p = np.minimum(bbox_p[0], bbox_p[2]).reshape(-1,1)
    x2p = np.maximum(bbox_p[0], bbox_p[2]).reshape(-1,1)
    y1p = np.minimum(bbox_p[1], bbox_p[3]).reshape(-1,1)
    y2p = np.maximum(bbox_p[1], bbox_p[3]).reshape(-1,1)

    bbox_p = np.concatenate([x1p, y1p, x2p, y2p], axis=0)

    # calc area of Bg
    area_p = (bbox_p[2] - bbox_p[0]) * (bbox_p[3] - bbox_p[1])
    # calc area of Bp
    area_g = (bbox_g[2] - bbox_g[0]) * (bbox_g[3] - bbox_g[1])

    # cal intersection
    x1I = np.maximum(bbox_p[0], bbox_g[0])
    y1I = np.maximum(bbox_p[1], bbox_g[1])
    x2I = np.minimum(bbox_p[2], bbox_g[2])
    y2I = np.minimum(bbox_p[3], bbox_g[3])
    I = np.maximum((y2I - y1I), 0) * np.maximum((x2I - x1I), 0)

    # find enclosing box
    x1C = np.minimum(bbox_p[0], bbox_g[0])
    y1C = np.minimum(bbox_p[1], bbox_g[1])
    x2C = np.maximum(bbox_p[2], bbox_g[2])
    y2C = np.maximum(bbox_p[3], bbox_g[3])

    # calc area of Bc
    area_c = (x2C - x1C) * (y2C - y1C)
    U = area_p + area_g - I
    iou = 1.0 * I / (U + 1e-16)

    # Giou
    giou = iou - (area_c - U) / (area_c + 1e-16)
    
    return giou


def assign_detections_to_trackers(trackers, detections, iou_thrd = 0.3):
    '''
    From current list of trackers and new detections, output matched detections,
    unmatchted trackers, unmatched detections.
    '''    
    
    IOU_mat= np.zeros((len(trackers),len(detections)),dtype=np.float32)
    for t,trk in enumerate(trackers):
        #trk = convert_to_cv2bbox(trk) 
        for d,det in enumerate(detections):
         #   det = convert_to_cv2bbox(det)
            trk = np.asarray(trk)
            det = np.asarray(det)
            IOU_mat[t,d] = Giou_np(trk,det) 
    # Produces matches       
    # Solve the maximizing the sum of IOU assignment problem using the
    # Hungarian algorithm (also known as Munkres algorithm)
    matched_idx = linear_assignment(-IOU_mat)        
    unmatched_trackers, unmatched_detections = [], []
    for t,trk in enumerate(trackers):
        if(t not in matched_idx[:,0]):
            unmatched_trackers.append(t)

    for d, det in enumerate(detections):
        if(d not in matched_idx[:,1]):
            unmatched_detections.append(d)

    matches = []
   
    # For creating trackers we consider any detection with an 
    # overlap less than iou_thrd to signifiy the existence of 
    # an untracked object

    for m in matched_idx:
        if(IOU_mat[m[0],m[1]]<iou_thrd):
            unmatched_trackers.append(m[0])
            unmatched_detections.append(m[1])
        else:
            matches.append(m.reshape(1,2))
    
    if(len(matches)==0):
        matches = np.empty((0,2),dtype=int)
    else:
        matches = np.concatenate(matches,axis=0)
    
    return matches, np.array(unmatched_detections), np.array(unmatched_trackers)   

from collections import deque

max_age = 2  # no.of consecutive unmatched detection before 
             # a track is deleted

min_hits = 1  # no. of consecutive matches needed to establish a track

tracker_list =[] # list for trackers
# list for track ID
#track_id_list= deque(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K'])
track_id_list= deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

def pipeline(img,zz_box):
    '''
    Pipeline function for detection and tracking
    '''
    global tracker_list
    global max_age
    global min_hits
    global track_id_list
    
    img_dim = (img.shape[1], img.shape[0])
    bz_box = zz_box 
    x_box =[]

    
    if len(tracker_list) > 0:
        for trk in tracker_list:
            x_box.append(trk.box)
    
    matched, unmatched_dets, unmatched_trks = assign_detections_to_trackers(x_box, bz_box, iou_thrd = -0.5) 


    # Deal with matched detections     
    if matched.size >0:
        for trk_idx, det_idx in matched:
            z = bz_box[det_idx]
            z = np.expand_dims(z, axis=0).T
            tmp_trk= tracker_list[trk_idx]
            tmp_trk.kalman_filter(z)
            xx = tmp_trk.x_state.T[0].tolist()
            xx =[xx[0], xx[2], xx[4], xx[6]]
            x_box[trk_idx] = xx
            tmp_trk.box =xx
            tmp_trk.hits += 1
            tmp_trk.no_losses = 0
    
    # Deal with unmatched detections      
    if len(unmatched_dets)>0:
        for idx in unmatched_dets:
            z = bz_box[idx]
            z = np.expand_dims(z, axis=0).T
            tmp_trk = Tracker() # Create a new tracker
            x = np.array([[z[0], 0, z[1], 0, z[2], 0, z[3], 0]]).T
            tmp_trk.x_state = x
            tmp_trk.predict_only()
            xx = tmp_trk.x_state
            xx = xx.T[0].tolist()
            xx =[xx[0], xx[2], xx[4], xx[6]]
            tmp_trk.box = xx
            tmp_trk.id = track_id_list.popleft() # assign an ID for the tracker
            tracker_list.append(tmp_trk)
            x_box.append(xx)
    
    # Deal with unmatched tracks       
    if len(unmatched_trks)>0:
        for trk_idx in unmatched_trks:
            tmp_trk = tracker_list[trk_idx]
            tmp_trk.no_losses += 1
            tmp_trk.predict_only()
            xx = tmp_trk.x_state
            xx = xx.T[0].tolist()
            xx =[xx[0], xx[2], xx[4], xx[6]]
            tmp_trk.box =xx
            x_box[trk_idx] = xx
                   
       
    # The list of tracks to be annotated  
    good_tracker_list =[]
    for trk in tracker_list:
        if ((trk.hits >= min_hits) and (trk.no_losses <=max_age)):
            good_tracker_list.append(trk)
            x_cv2 = trk.box
            ID = trk.id
            img= draw_boxes(img, boxes=[x_cv2],labels= [ID], class_name_map = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K']).astype(np.uint8) # Draw the bounding boxes on the 
                                             # images
    # Book keeping
    deleted_tracks = filter(lambda x: x.no_losses >max_age, tracker_list)  
    
    for trk in deleted_tracks:
            track_id_list.append(trk.id)
    
    tracker_list = [x for x in tracker_list if x.no_losses<=max_age]
    
    return img

/home/user/anaconda3/envs/chineseocr/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


# 进行跟踪 

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
# for i, img in enumerate(det_name):
#     boxes = z_box[i]
#     boxes = np.array(boxes, dtype=float)
#     zz_box =[boxes]
#     image_name = os.path.basename(img)
#     img = np.array(Image.open(img).convert("RGB"))

#     img = pipeline(img, zz_box)
#     Image.fromarray(img).save(os.path.join("./test_tracking", image_name))
# print("finish")

In [6]:
### For simple Kalman Filter ###
MAX_MISS = 12 #maximum miss number before a tracker being deleted
MIN_HIT  = 3 #minimum hit number for building a new tracker for tracking
miss_count = 0
hit_count  = 0
restart = False #the need of restarting a tracker
tracker  = Tracker()
tracker.x_state = [z_box[0][0] , 2 , z_box[0][1] , 2 , z_box[0][2] , 2 , z_box[0][3] , 2]
estimate = []
for idx, coor in enumerate(z_box):
    if not np.all(coor == [0,0,0,0]):
        if restart:
            if hit_count+1 == MIN_HIT:
                hit_count = 0
                tracker  = Tracker()
                tracker.x_state = [coor[0] , 2.5 , coor[1] , 5 , coor[2] , 2.5 , coor[3] , 2.5]
                restart = False
                state = tracker.x_state
                estimate.append([state[0], state[2], state[4], state[6]])
                continue
                
            else:
                hit_count += 1
                tracker.predict_only()
                estimate.append([0, 0, 0, 0])
                continue
        tracker.kalman_filter(coor)
        state = tracker.x_state
        estimate.append([state[0], state[2], state[4], state[6]])
    else:
        if not hit_count == 0:
            hit_count =0
        if miss_count < MAX_MISS and not restart:
            tracker.predict_only()
            state = tracker.x_state
            estimate.append([state[0], state[2], state[4], state[6]])
            miss_count += 1
        elif miss_count == MAX_MISS:
            restart = True
            miss_count = 0
            estimate.append([0, 0, 0, 0])
        else:
            estimate.append([0, 0, 0, 0])
print("end")

./image/frame_vc1_7078.bmp
./image/frame_vc1_7259.bmp
./image/frame_vc1_7326.bmp
./image/frame_vc1_7640.bmp
./image/frame_vc1_8063.bmp
end


In [7]:
for idx,img in enumerate(det_name):
    boxes = estimate[idx]
    boxes = np.array(boxes, dtype=float)
    image_name = os.path.basename(img)
    img = np.array(Image.open(img).convert("RGB"))
    
    ### write the image from Kalman filter for detection missing repair ###
    drawn_image = draw_boxes(img, [boxes], None, None, None).astype(np.uint8)
    Image.fromarray(drawn_image).save(os.path.join("./test_Kalman", image_name))
    
    ### write the image of the sort algorithm ###
    drawn_image = pipeline(img, [boxes])
    Image.fromarray(drawn_image).save(os.path.join("./test_tracking", image_name))